In [ ]:
%load_ext autoreload
%autoreload 2

## Dataset & Dataloader

In [ ]:
import os
os.chdir("../")

from omegaconf import OmegaConf

from neuralfp.data.datasets import MusicSegmentDataset, collate_data
from neuralfp.utils.common import load_dataset

In [ ]:
config = "configs/train.yaml"
config = OmegaConf.load(config)

dataset = MusicSegmentDataset(config["dataset"]["train"])

In [ ]:
from torch.utils.data import DataLoader

dataloader = DataLoader(
    dataset=dataset,
    collate_fn=collate_data,
    shuffle=False,
    **config["dataset"]["loaders"],
)

In [ ]:
# import tqdm

# for batch in tqdm.tqdm(dataloader):
#     features, targets = batch
#     print(features.shape, targets.shape)

## Model

In [ ]:
from neuralfp.model.neuralfp import NeuralAudioFingerprinter

In [ ]:
config = "configs/train.yaml"
config = OmegaConf.load(config)

model = NeuralAudioFingerprinter(**config["model"]["neuralfp"])

In [ ]:
import tqdm
import torch

for batch in tqdm.tqdm(dataloader):
    features, targets = batch
    print("features", features.shape)
    xs = torch.stack([features, targets], dim=0)
    print("xs", xs.shape)

    xs = torch.flatten(xs, 0, 1)
    out = model(xs)
    print("out", out.shape)


## Loss function

In [ ]:
from neuralfp.criterion.contrastive_loss import NTxentLoss

criterion = NTxentLoss()

In [ ]:
import tqdm
import torch

for batch in tqdm.tqdm(dataloader):
    features, targets = batch
    xs = torch.stack([features, targets], dim=0)
    xs = torch.flatten(xs, 0, 1)
    out = model(xs)
    n_anchors = out.shape[0] // 2
    print("n_anchors", n_anchors)
    loss = criterion(
        out[:n_anchors, :], out[n_anchors:, :], n_anchors
    )
    print("loss", loss)

## Load checkpoint

In [ ]:
import torch

checkpoint = torch.load("/home/huynd/Code/AI-beat-maker/train/artifacts/neuralfp_epoch88.pt", map_location="cpu")

In [ ]:
checkpoint["state_dict"]